# 1. Import module requirement

## 1.1 Import standard Python modules

In [2]:
from collections import OrderedDict, defaultdict

## 1.2 Import openalea packages


In [3]:
from openalea.deploy.shared_data import shared_data
from openalea.mtg import *
from openalea.mtg.algo import orders
from openalea.mtg import stat
import openalea.plantgl.all as pgl

NOT using graph editor observer No module named grapheditor


In [4]:
from openalea.core import path
from openalea.mtg import *
from openalea.deploy.shared_data import shared_data
import glob

## 1.3 Import strawberry modules

In [5]:
import openalea.strawberry

# rename Rules_production to geometry
from openalea.strawberry import Rules_production
# Replace Visualisation by visualisation 
from openalea.strawberry import Visualization
from openalea.strawberry import analysis

# TODO: Move functions into ???
#from openalea.strawberry import variables

# 2. Import data and read MTG in files

## 2.1 Import data

In [6]:
%gui qt4

files = shared_data(openalea.strawberry).glob('*.mtg')
#print files

genotypes = dict((f.namebase, f) for f in files)
#print(genotypes.keys())

DIPLOIDS = ['Sicile_New','Nils_New']
OCTOPLOIDS = ['Gariguette_NEW','Darselect_New','Clery_New','Capriss_New','Cir107_NEW','Ciflorette_New']

diploids = dict((k,f) for k,f in genotypes.items() if k in DIPLOIDS)
print(diploids)

octoploids = dict((k,f) for k,f in genotypes.items() if k in OCTOPLOIDS)
print(octoploids)

{u'Sicile_New': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Sicile_New.mtg'), u'Nils_New': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Nils_New.mtg')}
{u'Gariguette_NEW': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Gariguette_NEW.mtg'), u'Darselect_New': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Darselect_New.mtg'), u'Clery_New': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Clery_New.mtg'), u'Capriss_New': Path(u'c:\\users\\mlabadie\\appdat

## 2.2 Load one octoploid and one diploid  genotype 

A set of strawberry plant architecture has been measured and is stored in an MTG.

- We select on octoploid and/or one diploid genotype for architectural analysis.

In [7]:
#OCTOPLOIDS = ['Gariguette_NEW','Darselect_New','Clery_New','Capriss_New''Cir107_NEW','Ciflorette_New']
#DIPLOIDS = ['Sicile_New','Nils_New.mtg']

di_genotype = 'Sicile_New'
oct_genotype = 'Ciflorette_New'

gdi = MTG(diploids[di_genotype])
print('Diploid  : ', str(gdi))

goct = MTG(octoploids[oct_genotype])
print('Octoploid  : ', str(goct))

replace all the date format by -
('Diploid  : ', 'MTG : nb_vertices=4181, nb_scales=4')
replace all the date format by -
('Octoploid  : ', 'MTG : nb_vertices=6438, nb_scales=4')


* Check number of plant present in MTG

In [8]:
# Diploid
nb_plants = gdi.nb_vertices(scale=1)
print(nb_plants)

# Octoploid
nb_plants = goct.nb_vertices(scale=1)
print(nb_plants)

118
54


## 2.3 Load whole octoploids or diploids in one big MTG

A set of strawberry plant architecture has been measured and is stored in an MTG.

- We select whole octoploids genotype for architectural analysis.

In [9]:
big_mtg = MTG()
for fn in OCTOPLOIDS:
    big_mtg = algo.union(big_mtg,MTG(octoploids[fn]))
    g = big_mtg
    
genotypes = set(g.property("Genotype").itervalues())
print genotypes

replace all the date format by -
replace all the date format by -
replace all the date format by -
replace all the date format by -
replace all the date format by -
replace all the date format by -
set(['Ciflorette', 'Darselect', 'Gariguette', 'Capriss', 'Clery', 'Cir107'])


- We select whole diploids genotype for architectural analysis.

In [10]:
big_mtg = MTG()
for fn in DIPLOIDS:
    big_mtg = algo.union(big_mtg,MTG(diploids[fn]))
    g = big_mtg
    
genotypes = set(g.property("Genotype").itervalues())
print genotypes

replace all the date format by -
replace all the date format by -
set(['Nils', 'Sicile'])


# 3. Strawberry analysis

## 3.1 Function to run the notebook on diploid or octoploid (only for one genotype)

In [11]:
# Run the notebook on Diploid or Octoploid

#Choose false to read octoploid (and true to diploid)
IS_DIPLOID = False

if IS_DIPLOID:
    g = gdi
else:
    g = goct

# Dictionnary to convert propertie names in order to run old version script
convert = dict(Stade='Stade', 
               Fleurs_ouverte='FLWRNUMBER_OPEN',
               Fleurs_avorte='FLWRNUMBER_ABORTED',
               Fleurs_total='FLWRNUMBER',
               date='Sample_date',
               Plante='Plant_ID',
              )

if not IS_DIPLOID:
    g.properties()['Sample_date'] = g.property('Architecture_date')

# specific date of diploid data
def dates_diploid():
    l = '11-05-2016', '17-05-2016', '01-06-2016', '20-06-2016'
    return dict(zip(l, range(1, len(l)+1)))

#specific date for octoploide date
def dates_octoploid():
    d = {'10-12-2014':1,'07-01-2015':2,'15-02-2015':3,'02-03-2015':4,'03-04-2015':5,'27-05-2015':6,'04-12-2014':1,'13-02-2015':3,
         '30-03-2015':5,'08-01-2015':2,'11-02-2015':3,'04-03-2015':4,'02-04-2015':5,'20-05-2015':6,'09-01-2015':2,'06-03-2015':4,
         '12-02-2015':3,'19-05-2015':6}
    return d

def dates():
    if IS_DIPLOID:
        return dates_diploid()
    else: 
        return dates_octoploid()


def property(g, name):
    """ We can change the name of the MTG properties without changing the code"""
    return g.property(convert.get(name, name))

## 3.2 Data sequence extractions

In [ ]:
strawberry_data

# 4. Strawberry Visualisation

## 4.1 3D-Visualisation

In [12]:
g.properties()['order'] = orders(g)
Rules_production.color_code(g)
scene = Visualization.visualise_plants(g)

pgl.Viewer.display(scene)

c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:64: Warning: Invalid Color value 8 in setColor (maximum is 6)
  t.setColor(6+order)
c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:64: Warning: Invalid Color value 7 in setColor (maximum is 6)
  t.setColor(6+order)
c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:112: Warning: Invalid Color value 10 in setColor (maximum is 6)
  t.setColor(8+order)
c:\users\mlabadie\appdata\local\continuum\miniconda2\envs\strawberry\lib\site-packages\openalea.strawberry-1.0.0-py2.7.egg\openalea\strawberry\Rules_production.py:112: Warning: Invalid Color value 9 in setColor (maximum is 6)
  t.setColor(8+order)
c:\users\

## 4.2 2D-Vizualisation 